<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Task_Specific_Adapters_for_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer  # Ensure AutoTokenizer is imported

# Define the adapter module
class Adapter(nn.Module):
    def __init__(self, hidden_size, bottleneck_size):
        super().__init__()
        self.down_proj = nn.Linear(hidden_size, bottleneck_size)
        self.up_proj = nn.Linear(bottleneck_size, hidden_size)

    def forward(self, x):
        return x + self.up_proj(F.relu(self.down_proj(x)))

# Adding adapters to a transformer layer
class AdapterTransformer(nn.Module):
    def __init__(self, base_model, hidden_size=768, bottleneck_size=64):
        super().__init__()
        self.model = base_model
        self.adapter = Adapter(hidden_size, bottleneck_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        adapted_output = self.adapter(outputs)
        return adapted_output

# Example usage
# Load a base model from Hugging Face
base_model = AutoModel.from_pretrained("bert-base-uncased")

# Initialize the AdapterTransformer with the base model
adapter_model = AdapterTransformer(base_model)

# Example input
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
input_text = "This is an example sentence."
inputs = tokenizer(input_text, return_tensors="pt")

# Forward pass through the adapter model
output = adapter_model(inputs["input_ids"], inputs["attention_mask"])

print(output)